In [44]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE, SelectKBest, f_regression

from zipfile import ZipFile

**Чтение данных**

In [45]:
with ZipFile('data/data_ford_price.zip') as myzip:
    fd = pd.read_excel(myzip.open('data_ford_price.xlsx'))
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [46]:
# выбираем числовые признаки, разбиваем выборку на тренировочную и валидационную
num_columns = fd.select_dtypes(include=['int', 'float']).columns
data = fd[num_columns].copy()
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

**выделим три наиболее значимых признака методом RFE**

In [47]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'cylinders'], dtype=object)

**выделим три наиболее значимых признака методом SelectKBest**

In [48]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'odometer'], dtype=object)

**обучим модель на трех признаках от RFE и оценим качество обучения**

In [49]:
X_train_rfe = X_train[['year', 'condition', 'cylinders']]
X_test_rfe = X_test[['year', 'condition', 'cylinders']]

lr_rfe = LinearRegression()
lr_rfe.fit(X_train_rfe, y_train)
y_train_predicted_rfe = lr_rfe.predict(X_train_rfe)
y_test_predicted_rfe = lr_rfe.predict(X_test_rfe)
 
mae_train = mean_absolute_error(y_train, y_train_predicted_rfe)
mae_test = mean_absolute_error(y_test, y_test_predicted_rfe)
print('MAE train: %.3f' % mae_train)
print('MAE test:  %.3f' % mae_test)

MAE train: 5175.390
MAE test:  5011.071


**обучим модель на трех признаках от SelectKBest и оценим качество обучения**

In [50]:
X_train_sb = X_train[['year', 'condition', 'odometer']]
X_test_sb = X_test[['year', 'condition', 'odometer']]

lr_sb = LinearRegression()
lr_sb.fit(X_train_sb, y_train)
y_train_predicted_sb = lr_sb.predict(X_train_sb)
y_test_predicted_sb = lr_sb.predict(X_test_sb)
 
mae_train = mean_absolute_error(y_train, y_train_predicted_sb)
mae_test = mean_absolute_error(y_test, y_test_predicted_sb)
print('MAE train: %.3f' % mae_train)
print('MAE test:  %.3f' % mae_test)

MAE train: 4914.638
MAE test:  4802.231


Сравнив MAE на тестовых данных для моделей, обученных на трех лучших признаках от RFE и SelectKBest, можно сказать, что признаки, выбранные SelectKBest, показали лучшую из двух метрику MAE 4802 против 5011. Т.к. MAE - средняя абсолютная ошибка, лучшей считаем ту модель, которая показала меньшую ошибку.

Тем не менее, линейная регрессия, обученная на шести признаках в предыдущем юните, показала MAE на тестовых данных 4683, что, несомненно, более хороший результат.

Если исключить из рассмотрения параметр condition, то SelectKBest выделит три признака, обучение на которых даст MAE 4709.

И только обучение на 4 признаках, выделенных SelectKBest, даст MAE на тестовых данных немного лучше бейслайна - 4669.

**обучим модель на трех признаках от SelectKBest, исключая condition, и оценим качество обучения**

In [51]:
X_train_sbe = X_train[['year', 'cylinders', 'odometer']]
X_test_sbe = X_test[['year', 'cylinders', 'odometer']]

lr_sbe = LinearRegression()
lr_sbe.fit(X_train_sbe, y_train)
y_train_predicted_sbe = lr_sbe.predict(X_train_sbe)
y_test_predicted_sbe = lr_sbe.predict(X_test_sbe)
 
mae_train = mean_absolute_error(y_train, y_train_predicted_sbe)
mae_test = mean_absolute_error(y_test, y_test_predicted_sbe)
print('MAE train: %.3f' % mae_train)
print('MAE test:  %.3f' % mae_test)

MAE train: 4896.205
MAE test:  4708.946


**выделим четыре наиболее значимых признака методом SelectKBest**

In [52]:
selector = SelectKBest(f_regression, k=4)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'cylinders', 'odometer'], dtype=object)

**обучим модель на четырех признаках от SelectKBest и оценим качество обучения**

In [53]:
X_train_sb4 = X_train[['year', 'condition', 'cylinders', 'odometer']]
X_test_sb4 = X_test[['year', 'condition', 'cylinders', 'odometer']]

lr_sb4 = LinearRegression()
lr_sb4.fit(X_train_sb4, y_train)
y_train_predicted_sb4 = lr_sb4.predict(X_train_sb4)
y_test_predicted_sb4 = lr_sb4.predict(X_test_sb4)
 
mae_train = mean_absolute_error(y_train, y_train_predicted_sb4)
mae_test = mean_absolute_error(y_test, y_test_predicted_sb4)
print('MAE train: %.3f' % mae_train)
print('MAE test:  %.3f' % mae_test)

MAE train: 4815.000
MAE test:  4669.432
